#Parquet To Delta Lake

This pipeline is designed to ingest data from Parquet files and create a Delta Lake in the Bronze layer of the medallion architecture. The pipeline performs schema alignment and data partitioning, ensuring data is organized and accessible for downstream processing.

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

##1. Data Reading

Dynamic widgets are set up to specify the data partition by year and month, allowing flexible selection of time-based data for processing. This modular approach makes it easy to load only the relevant data, enhancing reusability.

In [0]:
dbutils.widgets.text("Year", "")
dbutils.widgets.text("Month", "")

partition_year = dbutils.widgets.get("Year")
partition_month = dbutils.widgets.get("Month")

In [0]:
parquet_df = spark.read.format("parquet").load(f"/mnt/bronze/parquet_data/PartitionYear={partition_year}/PartitionMonth={partition_month}")

##2. Delta Lake Schema Alignment

To ensure compatibility with the Delta Lake schema, columns are cast to specific data types. This step is crucial to maintain data consistency and compatibility for analysis in later stages.

In [0]:
parquet_df = (parquet_df
  .withColumn("VendorID", col("VendorID").cast(IntegerType()))
  .withColumn("tpep_pickup_datetime", col("tpep_pickup_datetime").cast(TimestampType())) 
  .withColumn("tpep_dropoff_datetime", col("tpep_dropoff_datetime").cast(TimestampType())) 
  .withColumn("passenger_count", col("passenger_count").cast(LongType()))
  .withColumn("trip_distance", col("trip_distance").cast(DoubleType()))
  .withColumn("RatecodeID", col("RatecodeID").cast(LongType()))
  .withColumn("store_and_fwd_flag", col("store_and_fwd_flag").cast(StringType()))
  .withColumn("PULocationID", col("PULocationID").cast(IntegerType()))
  .withColumn("DOLocationID", col("DOLocationID").cast(IntegerType())) 
  .withColumn("payment_type", col("payment_type").cast(LongType()))
  .withColumn("fare_amount", col("fare_amount").cast(DoubleType()))
  .withColumn("extra", col("extra").cast(DoubleType()))
  .withColumn("mta_tax", col("mta_tax").cast(DoubleType()))
  .withColumn("tip_amount", col("tip_amount").cast(DoubleType()))
  .withColumn("tolls_amount", col("tolls_amount").cast(DoubleType())) 
  .withColumn("improvement_surcharge", col("improvement_surcharge").cast(DoubleType())) 
  .withColumn("total_amount", col("total_amount").cast(DoubleType())) 
  .withColumn("congestion_surcharge", col("congestion_surcharge").cast(DoubleType())) 
  .withColumn("Airport_fee", col("Airport_fee").cast(DoubleType()))
  .withColumn("PartitionYear", lit(partition_year).cast(IntegerType())) 
  .withColumn("PartitionMonth", lit(partition_month).cast(IntegerType()))
)

parquet_df.printSchema()

##3. Data Writing

The transformed data is written to Delta format in the Bronze layer, partitioned by year and month. Partitioning optimizes data storage and querying, making it suitable for analysis and further transformation.

In [0]:
parquet_df.write.format("delta").mode("append").partitionBy("PartitionYear", "PartitionMonth").save("/mnt/bronze/delta_data/")